In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import os

##### 검색을 위한 데이터 준비

In [3]:
# 1. 임베더
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

# 2. 텍스트 스플리터
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=1000, chunk_overlap=200, encoding_name="cl100k_base"
)

# 3. Documents들 준비
# RAG_PDF 아래의 모든 pdf파일을 로드.
documents = []
base_dir = "../RAG_PDF"
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)  # RAG_PDF/JAVA_PDF
    if os.path.isdir(folder_path):  # 폴더인지 확인
        print(f":열린_파일_폴더: Processing folder: {folder_name}")
        # 현재 폴더내 pdf파일들 탐색
        for file_name in os.listdir(folder_path):
            if file_name.endswith(".pdf"):
                file_path = os.path.join(folder_path, file_name)
                print(f":글씨가_쓰여진_페이지: Loading: {file_name}")  # 체크
                # PDF 파일 로드
                loader = PyMuPDFLoader(file_path)
                document = loader.load_and_split(text_splitter=text_splitter)
                documents.extend(document)

# 4. Chroma db에 문서들 저장
chroma_db = Chroma.from_documents(
    documents=documents, embedding=embeddings, persist_directory="../chroma_rag_db"
)

:열린_파일_폴더: Processing folder: JAVA_PDF
:글씨가_쓰여진_페이지: Loading: 10_다형성.pdf
:글씨가_쓰여진_페이지: Loading: 11_기본API.pdf
:글씨가_쓰여진_페이지: Loading: 12_예외처리(Exception).pdf
:글씨가_쓰여진_페이지: Loading: 13_입출력(IO).pdf
:글씨가_쓰여진_페이지: Loading: 14-1_컬렉션(Collection).pdf
:글씨가_쓰여진_페이지: Loading: 14-2_제네릭스(Generics).pdf
:글씨가_쓰여진_페이지: Loading: 15_스레드.pdf
:글씨가_쓰여진_페이지: Loading: 18_네트워크(Network).pdf
:글씨가_쓰여진_페이지: Loading: 1_프로그래밍기초.pdf
:글씨가_쓰여진_페이지: Loading: 2_변수(Variable).pdf
:글씨가_쓰여진_페이지: Loading: 3_연산자(Operator).pdf
:글씨가_쓰여진_페이지: Loading: 4_제어문.pdf
:글씨가_쓰여진_페이지: Loading: 5_배열.pdf
:글씨가_쓰여진_페이지: Loading: 6_2차원배열.pdf
:글씨가_쓰여진_페이지: Loading: 7_객체.pdf
:글씨가_쓰여진_페이지: Loading: 8_객체배열.pdf
:글씨가_쓰여진_페이지: Loading: 9_상속.pdf
:열린_파일_폴더: Processing folder: JDBC_PDF
:글씨가_쓰여진_페이지: Loading: 1_JDBC개요.pdf
:글씨가_쓰여진_페이지: Loading: 2_JDBC실습구조.pdf
:열린_파일_폴더: Processing folder: JQ_PDF
:글씨가_쓰여진_페이지: Loading: 0_개요.pdf
:글씨가_쓰여진_페이지: Loading: 1_객체탐색.pdf
:글씨가_쓰여진_페이지: Loading: 2_객체조작.pdf
:글씨가_쓰여진_페이지: Loading: 3_이벤트.pdf
:글씨가_쓰여진_페이지: Loading: 4_Effect

In [ ]:
# similarity 기반
retriever = chroma_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k":3}
)

docs = retriever.get_relevant_documents("JVM에 대해 알려줘")
docs

C:\Users\user1\AppData\Local\Temp\ipykernel_572\97103156.py:6: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents("JVM에 대해 알려줘")


[Document(id='ab0d83e4-6e32-4cf4-908e-519b50a45d7c', metadata={'total_pages': 26, 'source': 'RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'author': 'user1', 'format': 'PDF 1.4', 'creator': 'Hancom PDF 1.3.0.538', 'page': 4, 'file_path': 'RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'keywords': '', 'trapped': '', 'creationDate': "D:20210107085705+09'00'", 'moddate': '2021-01-07T08:57:05+09:00', 'subject': '', 'producer': 'Hancom PDF 1.3.0.538', 'title': '', 'modDate': "D:20210107085705+09'00'", 'creationdate': '2021-01-07T08:57:05+09:00'}, page_content='▶JVM(Java Virtual Machine)\n자바를실행하기위한가상기계로플랫폼에의존적\nbyte code(class파일)를해석하고실행하는interpreter'),
 Document(id='5caaef52-7906-4ab1-b96d-e45507cac0cd', metadata={'author': 'user1', 'modDate': "D:20210107085705+09'00'", 'creationdate': '2021-01-07T08:57:05+09:00', 'page': 4, 'trapped': '', 'creationDate': "D:20210107085705+09'00'", 'source': '../RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'total_pages': 26, 'creator': 'Hancom PDF 1.3.0.538', 'producer': 'Hancom PDF 1.3.0.5

In [6]:
print(docs[0])

page_content='▶JVM(Java Virtual Machine)
자바를실행하기위한가상기계로플랫폼에의존적
byte code(class파일)를해석하고실행하는interpreter' metadata={'total_pages': 26, 'source': 'RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'author': 'user1', 'format': 'PDF 1.4', 'creator': 'Hancom PDF 1.3.0.538', 'page': 4, 'file_path': 'RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'keywords': '', 'trapped': '', 'creationDate': "D:20210107085705+09'00'", 'moddate': '2021-01-07T08:57:05+09:00', 'subject': '', 'producer': 'Hancom PDF 1.3.0.538', 'title': '', 'modDate': "D:20210107085705+09'00'", 'creationdate': '2021-01-07T08:57:05+09:00'}


In [ ]:
# mmr 기반
retriever = chroma_db.as_retriever(
    search_type="mmr",
    search_kwargs={"k":5, "fetch_k":50}
)

docs = retriever.get_relevant_documents("JVM에 대해 알려줘")
docs

[Document(id='ab0d83e4-6e32-4cf4-908e-519b50a45d7c', metadata={'file_path': 'RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'source': 'RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'author': 'user1', 'creationDate': "D:20210107085705+09'00'", 'creationdate': '2021-01-07T08:57:05+09:00', 'total_pages': 26, 'trapped': '', 'producer': 'Hancom PDF 1.3.0.538', 'keywords': '', 'title': '', 'page': 4, 'creator': 'Hancom PDF 1.3.0.538', 'format': 'PDF 1.4', 'subject': '', 'moddate': '2021-01-07T08:57:05+09:00', 'modDate': "D:20210107085705+09'00'"}, page_content='▶JVM(Java Virtual Machine)\n자바를실행하기위한가상기계로플랫폼에의존적\nbyte code(class파일)를해석하고실행하는interpreter'),
 Document(id='2315ce2e-5e68-40f3-93e6-fcad2c62fd2a', metadata={'source': 'RAG_PDF\\JAVA_PDF\\7_객체.pdf', 'subject': '', 'creator': 'Hancom PDF 1.3.0.538', 'total_pages': 51, 'moddate': '2021-01-20T10:16:41+09:00', 'modDate': "D:20210120101641+09'00'", 'trapped': '', 'page': 29, 'producer': 'Hancom PDF 1.3.0.538', 'keywords': '', 'creationdate': '2021-01-20T10:16:41+0

In [8]:
print(docs[0])

page_content='▶JVM(Java Virtual Machine)
자바를실행하기위한가상기계로플랫폼에의존적
byte code(class파일)를해석하고실행하는interpreter' metadata={'file_path': 'RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'source': 'RAG_PDF\\JAVA_PDF\\1_프로그래밍기초.pdf', 'author': 'user1', 'creationDate': "D:20210107085705+09'00'", 'creationdate': '2021-01-07T08:57:05+09:00', 'total_pages': 26, 'trapped': '', 'producer': 'Hancom PDF 1.3.0.538', 'keywords': '', 'title': '', 'page': 4, 'creator': 'Hancom PDF 1.3.0.538', 'format': 'PDF 1.4', 'subject': '', 'moddate': '2021-01-07T08:57:05+09:00', 'modDate': "D:20210107085705+09'00'"}


##### 검색결과를 프롬프트에 추가하여 LLM에게 질문하기

In [9]:
# Prompt
template = """
Answer the question based only on the following context:
{context}
Question : {question}
"""

from langchain_openai import OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(template)
llm = OpenAI()
chain = prompt | llm | StrOutputParser()

def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

response = chain.invoke({
    "context": format_docs(docs),
    "question": "JVM에 대해 알려줘"
})

print(response)

Answer: JVM은 자바를 실행하기 위한 가상 기계로, 플랫폼에 의존적이며 byte code를 해석하고 실행하는 interpreter이다. 필드는 초기화 순서에 따라 인스턴스 변수, 클래스 변수, 클래스 초기화 블록 순으로 초기화되며, 인스턴스는 생성자를 통해 초기값을 설정할 수 있다. 또한 JVM은 기본값, 명시적 초기값, 클래스 초기화 블록을 통해 초기화될 수 있다. 자바를 설치할 때는 모두 Default로 설정하고, 폴더를 변경하면 해당 위치를 기억해야 한다. 


In [10]:
response = chain.invoke({
    "context": format_docs(docs),
    "question": "Spring에 대해 알려줘"
})

print(response)


Spring은 자바 기반의 오픈 소스 애플리케이션 프레임워크로서, 웹 애플리케이션 개발을 위한 다양한 기능을 제공합니다. 이를 통해 개발자는 복잡한 웹 애플리케이션을 빠르고 쉽게 구축할 수 있습니다. Spring은 다양한 모듈로 구성되어 있으며, 이를 조합하여 필요한 기능을 선택적으로 사용할 수 있습니다. 또한 Spring은 큰 규모의 애플리케이션 개발에도 유연하고 확장 가능한 구조를 제공합니다. 이를 통해 생산성을 향상시키고 유지보수를 용이하게 만듭니다. 


##### 간단한 RAG 실습

In [19]:
# 1. Doucment Load(Web)
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    "https://organization-bht.gitbook.io/khmkm/llm/1-llm/",
    encoding="utf-8",
    header_template={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36"
    },
)

# 2. Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = loader.load_and_split(text_splitter)

# 3. Embedding
from langchain_openai import OpenAIEmbeddings

openai_embedding = OpenAIEmbeddings()

# 4. Vector Store
from langchain_chroma import Chroma

db = Chroma.from_documents(documents=docs, embedding=openai_embedding)

# 5. retriever
retriever = db.as_retriever()

# 6. LLM 정의
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4o-mini")

##### MultiQueryRetriever

In [20]:
# 7. MultiQueryRetriever
# - 사용자의 질문을 다각도로 해석하여 여러개의 질문을 생성.
# - 여러개의 질문으로 벡터 스토어에서 검색 후 검색 결과를 결합한 프롬프트를 생성.

from langchain.retrievers.multi_query import MultiQueryRetriever

multiquery_retriever = MultiQueryRetriever.from_llm(
    retriever=retriever,
    llm=llm,
)

# 8. retrieval
question = "LLM?"
relevant_docs = multiquery_retriever.invoke(question)

print("======== 검색된 문서 갯수 ==========", len(relevant_docs))
relevant_docs

======== 검색된 문서 갯수 ========== 8


[Document(id='54708d40-4c64-4990-b733-f200002afd28', metadata={'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/', 'language': 'en', 'title': '1장 LLM에 대한 이해 | 경민민 IT 핸드북'}, page_content='시스템을 탄생시켰습니다. 이 가운데 LLM(Large Language Model)은 텍스트 데이터를 기반으로 언어를 이해하고 생성할 수 있는 가장 혁신적인 도구로 주목받고 있습니다이 장에서는 LLM의 개념, 작동 원리, 주요 특징, 그리고 활용 가능성을 살펴보겠습니다.1. LLM(Large Language Model)Large(엄청나게 많은 데이터를 학습한) Language(인간의 언어를 처리하는) Model(딥러닝모델)LLM(Large Language Model)은 대규모 언어 모델을 의미하며,  인터넷, 책, 논문, 뉴스, 블로그 등 방대한 양의 텍스트 데이터를 학습한 인공지능 모델입니다. 이러한 데이터들은 트랜스포머 아키텍쳐기반의 모델들에 학습되어 학습한 데이터의 문맥과 패턴을 이해할 수 있습니다. 이로 인해 LLM은 사람의 질문에도 이해를 하고 사람처럼 소통할 수 있는 능력을 갖추게 되었습니다. 사람이 사용하는 언어를 기계가 처리하는 기술을 NLP(Natural'),
 Document(id='535fd0dc-3b3b-428d-a5dd-c922c8dbca0d', metadata={'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/', 'title': '1장 LLM에 대한 이해 | 경민민 IT 핸드북', 'language': 'en'}, page_content='시스템을 탄생시켰습니다. 이 가운데 LLM(Large Language Model)은 텍스트 데이터를 기반으로 언어를 이해하고 생성할 수 있는 가장 혁신적인 도구로 주목받고 있습니다이 

In [21]:
print(relevant_docs[0].page_content)

시스템을 탄생시켰습니다. 이 가운데 LLM(Large Language Model)은 텍스트 데이터를 기반으로 언어를 이해하고 생성할 수 있는 가장 혁신적인 도구로 주목받고 있습니다이 장에서는 LLM의 개념, 작동 원리, 주요 특징, 그리고 활용 가능성을 살펴보겠습니다.1. LLM(Large Language Model)Large(엄청나게 많은 데이터를 학습한) Language(인간의 언어를 처리하는) Model(딥러닝모델)LLM(Large Language Model)은 대규모 언어 모델을 의미하며,  인터넷, 책, 논문, 뉴스, 블로그 등 방대한 양의 텍스트 데이터를 학습한 인공지능 모델입니다. 이러한 데이터들은 트랜스포머 아키텍쳐기반의 모델들에 학습되어 학습한 데이터의 문맥과 패턴을 이해할 수 있습니다. 이로 인해 LLM은 사람의 질문에도 이해를 하고 사람처럼 소통할 수 있는 능력을 갖추게 되었습니다. 사람이 사용하는 언어를 기계가 처리하는 기술을 NLP(Natural


##### 프롬프트 정의

In [22]:
# 10. 프롬프트 정의
template = """
Answer the question based only on the following context:
{context}
Question : {question}
"""
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(template)
from langchain_core.output_parsers import StrOutputParser
# Chain
chain = prompt | llm | StrOutputParser()
# 11 검색내용을 prompt에 추가하여 보강
response = chain.invoke({'context': (format_docs(relevant_docs)) , "question": question})
print("############# GPT 응답 ############")
print(response)

############# GPT 응답 ############
LLM(Large Language Model)은 대규모 언어 모델을 의미하며, 인터넷, 책, 논문, 뉴스, 블로그 등 방대한 양의 텍스트 데이터를 학습한 인공지능 모델입니다. LLM은 트랜스포머 아키텍처 기반으로 작동하며, 학습한 데이터의 문맥과 패턴을 이해하여 사람의 질문에 답하고 사람처럼 소통할 수 있는 능력을 갖추고 있습니다. 이 기술은 자연어 처리(NLP) 분야에서 활용되며, 챗봇, 텍스트 번역, 요약 등 다양한 분야에서 사람을 대체하거나 보조하는 역할을 합니다.


##### Contextual Compression Retriever

In [23]:
# 8. Contextual Compression Retriever
# - 검색된 문서들 중 쿼리와 관련된 정보만 압축하는 검색기.
# - 쿼리와 무관한 정보를 삭제해 줌으로써 응답 품질을 향상.

retriever = db.as_retriever(search_type="mmr", search_kwargs={"k":5, "fetch_k":20, "lambda_mult":0.25})

docs = retriever.get_relevant_documents(question)
print(question)

LLM?


In [24]:
print(len(docs))

5


In [25]:
docs

[Document(id='f695d4d2-ec66-4568-9318-96c6235b5489', metadata={'title': '1장 LLM에 대한 이해 | 경민민 IT 핸드북', 'language': 'en', 'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/'}, page_content='LLM은 잘하는 것과 못하는 것이 혼재LLM은 다양한 응답을 생성할 수 있지만 , 기본적으로 확률에 의거한 응답을 반환하기 때문에 잘못된 응답일 가능성이 존재합니다.혹은 소설, 시, 광고 문구 작성 등 단순 검색 기반이 아닌 창의적인 답변 생성이 가능합니다.단, 요청한 내용과 상관이 없는 잘못된 정보를 반환할 수도 있습니다.(Hallucination)LLM은 언어 패턴을 기반으로 응답하도록 설계되었기 때문에, 복잡한 수학계산이나 논리적 추론에서 오류를 범할 수 있습니다. 다양한 수학문제 해결을 위해서는 외부 계산 API 시스템과 연동하도록 설정해 줘야 합니다.출처 : https://www.linkedin.com/posts/kaicheng-yang-43477213b_have-been-thinking-about-what-tasks-llms-activity-7161062110209146880-pHDn3) 할루시네이션(Hallucination) - 환각Hallucination이란,'),
 Document(id='0ec7bce7-6e1a-4f0a-a77f-5cfd12bf1a47', metadata={'language': 'en', 'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/', 'title': '1장 LLM에 대한 이해 | 경민민 IT 핸드북'}, page_content='LLM도입시에는 프라이버시 문제, 정보의 정확성문제, 윤리적 문제 , 경제적 문제 등 고려해야할  부분들이 상당합니다.  이번 커리큘럼에서는 총 4장의 대단원에 걸쳐 이러한 문제

In [26]:
# 10. 검색결과 압축
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)
compressed_docs = compression_retriever.get_relevant_documents(question)
print(len(compressed_docs))

4


In [27]:
compressed_docs

[Document(metadata={'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/', 'title': '1장 LLM에 대한 이해 | 경민민 IT 핸드북', 'language': 'en'}, page_content='LLM은 잘하는 것과 못하는 것이 혼재LLM은 다양한 응답을 생성할 수 있지만 , 기본적으로 확률에 의거한 응답을 반환하기 때문에 잘못된 응답일 가능성이 존재합니다.혹은 소설, 시, 광고 문구 작성 등 단순 검색 기반이 아닌 창의적인 답변 생성이 가능합니다.단, 요청한 내용과 상관이 없는 잘못된 정보를 반환할 수도 있습니다.(Hallucination)LLM은 언어 패턴을 기반으로 응답하도록 설계되었기 때문에, 복잡한 수학계산이나 논리적 추론에서 오류를 범할 수 있습니다. 다양한 수학문제 해결을 위해서는 외부 계산 API 시스템과 연동하도록 설정해 줘야 합니다.'),
 Document(metadata={'title': '1장 LLM에 대한 이해 | 경민민 IT 핸드북', 'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/', 'language': 'en'}, page_content='LLM은 "딥러닝(Deep Learning)" 기술 중 트랜스포머(Transformer)라는 신경망 구조를 기반으로 만들어 졌습니다.  트랜스포머는 Self-Attention Mechanism을 통해 문장 내 단어들 사이의 관계를 파악하고, 문맥적 의미를 계산합니다. 이러한 구조 덕분에 LLM은 대규모 텍스트 데이터를 학습하며 문법, 언어 패턴, 사실 기반 지식까지 "통계적"으로 학습할 수 있게 됩니다.'),
 Document(metadata={'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/', 'language': 'en', 'title': '1장 LLM에 대한 이해 | 경민민

In [28]:
response = chain.invoke({'context': (format_docs(compressed_docs)) , "question": question})
print("############# GPT 응답 ############")
print(response)

############# GPT 응답 ############
LLM은 "Large Language Model"의 약자로, 대규모 텍스트 데이터를 기반으로 언어를 이해하고 생성할 수 있는 인공지능 시스템입니다. LLM은 트랜스포머라는 신경망 구조를 기반으로 하며, Self-Attention Mechanism을 통해 문장 내 단어들 사이의 관계를 파악하고 문맥적 의미를 계산합니다. 이 모델은 다양한 응답을 생성할 수 있지만, 확률에 의거한 응답을 반환하기 때문에 잘못된 정보나 오류가 발생할 가능성이 있습니다. LLM은 챗봇, 텍스트 번역, 요약 등 다양한 자연어 처리(NLP) 작업에 활용되며, 현대의 기업에서 AI 기능을 추가하는 데 필수적인 기술로 자리잡고 있습니다.


##### Reranker

In [30]:
# Reranker
# - RAG의 마지막 단계에서 수행되며, 검색결과 내용을 다시 평가하여 좀 더 정확한 응답을 반환받기 위해 사용

from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
# Reranker모델 초기화
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")

c:\LangchainWorkspace\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\LangchainWorkspace\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user1\.cache\huggingface\hub\models--BAAI--bge-reranker-v2-m3. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, se

In [31]:
compressor = CrossEncoderReranker(model=model, top_n=3)
compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=retriever)

compressed_docs = compression_retriever.invoke("llm")
print(len(compressed_docs))

3


In [32]:
response = chain.invoke({'context': (format_docs(compressed_docs)) , "question": question})
print("############# GPT 응답 ############")
print(response)

############# GPT 응답 ############
LLM은 "Large Language Model"의 약자로, 자연어 처리(NLP) 기술을 기반으로 하여 사람과 자연스럽게 대화하거나 복잡한 언어 작업을 수행할 수 있는 인공지능 모델입니다. LLM은 딥러닝 기술 중 트랜스포머라는 신경망 구조를 사용하여 문장 내 단어들 사이의 관계를 파악하고 문맥적 의미를 계산합니다. 이를 통해 대규모 텍스트 데이터를 학습하며 문법, 언어 패턴, 사실 기반 지식을 통계적으로 학습할 수 있습니다. LLM은 다양한 특징을 가지고 있으며, 사용 시 프라이버시, 정보의 정확성, 윤리적 문제, 경제적 문제 등을 고려해야 합니다.


In [33]:
compressed_docs

[Document(id='b1c189f3-2a11-460d-bf4b-7f958c5c62f3', metadata={'language': 'en', 'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/', 'title': '1장 LLM에 대한 이해 | 경민민 IT 핸드북'}, page_content='재평가자RAGPowered by GitBookOn this pageIntro1. LLM(Large Language Model)2. LLM 작동원리3. LLM의 특징1) NLP처리에 특화2) LLM은 잘하는 것과 못하는 것이 혼재3) 할루시네이션(Hallucination) - 환각4) LLM은 정적모델 5)  LLM의 크기와 성능의 관계6)  LLM은 편향적4. 대표 LLM 서비스1. GPT-42.  GPT-3.53. Claude 3 Opus4. Google DeepMind5. Amazon Olympus6. Meta LLMA27. X Grok CopyLLM 서비스1장 LLM에 대한 이해출처: https://www.darkreading.com/vulnerabilities-threats/top-lessons-cisos-owasp-llm-top-10Intro최근 인공지능(AI)의 발달은 자연어 처리(NLP) 기술을 통해 사람과 자연스럽게 대화하거나 복잡한 언어 작업을 수행할 수 있는'),
 Document(id='0ec7bce7-6e1a-4f0a-a77f-5cfd12bf1a47', metadata={'title': '1장 LLM에 대한 이해 | 경민민 IT 핸드북', 'language': 'en', 'source': 'https://organization-bht.gitbook.io/khmkm/llm/1-llm/'}, page_content='LLM도입시에는 프라이버시 문제, 정보의 정확성문제, 윤리적 문제 , 경제적 문제 등 고려해야할  부분들이 상당합니다.  이번 커리큘럼에서는 총 4장의 대단원에 걸쳐 이러한 문